# Задание
Получите данные по энергопотреблению первых 20 зданий (building_id от 0 до 19).

Заполните отсутствующие значения по погоде интерполяционными данными.

Разделите данные на обучающие/проверочные в пропорции 80/20.

Постройте (1) первый набор моделей линейной регрессии по часам для каждого из первых 20 зданий по следующим параметрам: air_temperature, dew_temperature, cloud_coverage, wind_speed, sea_level_pressure.

Постройте для этих же 20 зданий (2) второй набор моделей линейной регрессии по часам по параметрам: дни недели и праздники (is_holiday). Требуется построить еще 480 моделей.

Используйте логарифм целевого показателя (meter_reading_log) для обоих наборов моделей.

Данные:
* video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* video.ittensive.com/machine-learning/ashrae/train.0.csv.gz

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar


def reduce_mem_usage(df: pd.DataFrame):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo('f2').min and c_max < np.finfo('f2').max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo('f4').min and c_max < np.finfo('f4').max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == 'int':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == 'timestamp':
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != 'datetime':
            df[col] = df[col].astype('category')
    
    end_mem = df.memory_usage().sum() / 1024**2
    print(
        'Потребление памяти меньше на ',
        round(start_mem - end_mem, 2),
        ' Мб (-',
        round(100 * (start_mem - end_mem) / start_mem, 1),
        '%)',
        sep=''
    )
    return df


def show_inf_and_na(df: pd.DataFrame):
    pd.set_option('use_inf_as_na', True)
    for col in df.columns:
        print(col, 'Inf+NaN:', df[col].isnull().sum())
        

def generate_hourly_models(df: pd.DataFrame):
    '''
    Генерирует набор моделей линейной регрессии.
    Количество моделе соответствует колчеству часов представленных в df['timestamp']
    df['timestamp'] - ОБЯЗАТЕЛЬНЫЙ столбец во фрейме данных
    df[0:-1] - эти столбцы используются в качестве входных данных (факторных переменных) при обучении моделей
    f[-1] - последний столбец используется в качестве целевой переменной при обучении моделей
    '''
    hourly_models = dict()
    for hour in range(0, 24):
        hour_data = df[df['timestamp'].dt.hour == hour]
        hourly_models[hour] = LinearRegression().fit(
            hour_data.iloc[:, :-1].drop('timestamp', axis='columns').values,
            hour_data.iloc[:, -1].values
        )
    return hourly_models


def models_amount(models):
    amount = 0
    for i in range(len(models)):
        amount += len(models[i])
    return amount


buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")

### Получите данные по энергопотреблению первых 20 зданий (building_id от 0 до 19).

In [2]:
energy = energy[energy['building_id'] < 20]

### Заполните отсутствующие значения по погоде интерполяционными данными.

In [3]:
energy = pd.merge(left=energy, right=buildings, how='left', left_on='building_id', right_on='building_id')
energy = pd.merge(
    left=energy.set_index(['timestamp', 'site_id']), 
    right=weather.set_index(['timestamp', 'site_id']),
    how='left', left_index=True, right_index=True
)
energy.reset_index(inplace=True)
del weather
del buildings
energy = energy[energy['meter_reading'] > 0]
energy["precip_depth_1_hr"] = energy["precip_depth_1_hr"].map(lambda x: x if x > 0 else 0)

columns_for_interpolation = [
    'air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction'
]
for column in columns_for_interpolation:
    energy[column] = energy[column].interpolate(limit_direction='both', kind='cubic')
    
energy['meter_reading_log'] = np.log(energy['meter_reading'] + 1)


show_inf_and_na(energy[columns_for_interpolation])
energy = reduce_mem_usage(energy)

air_temperature Inf+NaN: 0
cloud_coverage Inf+NaN: 0
dew_temperature Inf+NaN: 0
precip_depth_1_hr Inf+NaN: 0
sea_level_pressure Inf+NaN: 0
wind_direction Inf+NaN: 0
Потребление памяти меньше на 9.53 Мб (-63.9%)


### Разделите данные на обучающие/проверочные в пропорции 80/20.
#### (1) первый набор

In [4]:
features_for_set1 = [
    'building_id', 'timestamp', # staff features
    'air_temperature', 'dew_temperature', 'cloud_coverage', 'wind_speed', 'sea_level_pressure', 'meter_reading_log' # features for predictions
]
energy_train_set1, energy_test_set1 = train_test_split(energy[features_for_set1], test_size=.2)
print(energy_train_set1.shape, energy_test_set1.shape)
energy_train_set1.head()

(86858, 8) (21715, 8)


,building_id,timestamp,air_temperature,dew_temperature,cloud_coverage,wind_speed,sea_level_pressure,meter_reading_log
123061,1,2016-09-13 09:00:00,24.406250,23.296875,4.542969,3.599609,1013.5,4.359375
73902,2,2016-06-02 23:00:00,30.000000,21.093750,5.410156,7.699219,1015.5,2.675781
150422,2,2016-11-09 09:00:00,17.796875,15.000000,4.851562,1.500000,1017.5,3.289062
145792,12,2016-10-30 17:00:00,30.000000,17.796875,4.000000,5.101562,1019.0,5.644531
78200,0,2016-06-11 22:00:00,29.406250,21.703125,6.000000,5.101562,1019.5,5.542969


####  (2) второй набор

In [5]:
onehot_feature_names_for_week_day = ['wd_' + str(i) for i in range(0, 7)]

energy = pd.merge(
    left=energy, 
    right=pd.get_dummies(
        energy['timestamp'].dt.weekday.astype('int8')
    )pd
)
us_holidays = calendar().holidays(start=energy['timestamp'].dt.date.min(), end=energy['timestamp'].dt.date.max())
energy['is_holiday'] = energy['timestamp'].isin(us_holidays).astype('int8')

features_for_set2 = [
    'building_id', 'timestamp', # staff features
]
features_for_set2.extend(onehot_feature_names_for_week_day)
features_for_set2.append('is_holiday')
features_for_set2.append('meter_reading_log')

energy_train_set2, energy_test_set2 = train_test_split(energy[features_for_set2], test_size=.2)
print(energy_train_set2.shape, energy_test_set2.shape)
energy_train_set2.head()

(86858, 11) (21715, 11)


,building_id,timestamp,wd_0,wd_1,wd_2,wd_3,wd_4,wd_5,wd_6,is_holiday,meter_reading_log
140576,16,2016-10-19 20:00:00,0,0,1,0,0,0,0,0,7.082031
161084,4,2016-12-01 14:00:00,0,0,0,1,0,0,0,0,7.429688
76587,7,2016-06-08 13:00:00,0,0,1,0,0,0,0,0,6.382812
136216,16,2016-10-10 18:00:00,1,0,0,0,0,0,0,0,7.109375
150526,6,2016-11-09 14:00:00,0,0,1,0,0,0,0,0,4.996094


### Постройте (1) первый набор моделей линейной регрессии по часам для каждого из первых 20 зданий по следующим параметрам: air_temperature, dew_temperature, cloud_coverage, wind_speed, sea_level_pressure.

In [6]:
set1_models = dict()
for building_id in range(0, energy['building_id'].max() + 1):    
    set1_models[building_id] = generate_hourly_models(
        energy_train_set1[energy_train_set1['building_id'] == building_id].drop('building_id', axis='columns')
    )


print('Всего моделей:', models_amount(set1_models))

Всего моделей: 480


### Постройте для этих же 20 зданий (2) второй набор моделей линейной регрессии по часам по параметрам: дни недели и праздники (is_holiday). Требуется построить еще 480 моделей.

In [7]:
set2_models = dict()
for building_id in range(0, energy['building_id'].max() + 1):    
    set2_models[building_id] = generate_hourly_models(
        energy_train_set2[energy_train_set2['building_id'] == building_id].drop('building_id', axis='columns')
    )


print('Всего моделей:', models_amount(set2_models))

Всего моделей: 480


### Какая модель, 1 или 2, оказалась точнее для первого здания (building_id = 0) ?

In [8]:
def squared_error(set_of_models, data):
    prediction = 0
    if data.building_id in set_of_models:
        if data.timestamp.hour in set_of_models[data.building_id]:
            prediction =  set_of_models[data.building_id][data.timestamp.hour].predict(data[2:-1].values.reshape(1, -1))[0]
    return (data[-1] - prediction)**2


def calculate_squared_error_for_set1(row):
    return squared_error(set1_models, row)


def calculate_squared_error_for_set2(row):
    return squared_error(set2_models, row)


errors1 = energy_test_set1[energy_test_set1['building_id'] == 0].apply(calculate_squared_error_for_set1, axis='columns')
errors2 = energy_test_set2[energy_test_set2['building_id'] == 0].apply(calculate_squared_error_for_set2, axis='columns')

error1 = np.sqrt(errors1.sum() / len(errors1))
error2 = np.sqrt(errors2.sum() / len(errors2))
print('Ошибка модели 1:', error1, '\tОшибка модели 2:', error2)
if error1 < error2:
    print('Модель 1 - точнее.')
elif error2 < error1:
    print('Модель 2 - точнее.')
else:
    print('Модели имеют одинаковую точность.')

Ошибка модели 1: 0.19685699663710124 	Ошибка модели 2: 0.2638724562883291
Модель 1 - точнее.
